Generate Cubes

In [1]:
#Import libraries

import ee 
import geemap
import xee
import xarray as xr
import datetime
import rioxarray as rxr
import pandas as pd
import matplotlib.pyplot as plt
import os
import ipyleaflet
import gdown
import geopandas as gpd
import rasterio
from rasterio import features
import numpy as np
from osgeo import gdal, ogr, osr


local_dir = "C:/Users/jaden/Downloads/Research/Notebooks/GEE/tifs"
os.makedirs(local_dir, exist_ok=True)

gdal.UseExceptions()

In [2]:
#Authenticate/initialize earth engine
ee.Authenticate()


True

In [4]:
ee.Initialize(
    project = 'sar-testing-472822',
    opt_url = 'https://earthengine-highvolume.googleapis.com'
)

*** Earth Engine *** Share your feedback by taking our Annual Developer Satisfaction Survey: https://google.qualtrics.com/jfe/form/SV_7TDKVSyKvBdmMqW?ref=4i2o6


In [2]:
Map = geemap.Map(basemap = 'TERRAIN')
Map #to view map, selected point in US

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], position='topright', transp…

Set bounding box in map widget above: for testing purposes, kennicott area
Note that the GRD image selection will include any scenes that *intersect* the box bounds

In [3]:
# Get the last drawn feature
loc = Map.draw_last_feature.geometry()

In [4]:
#Get GRD images
path = 14 #set path, same as ASF number
frame = 5 #set frame [WIP], requires trial and error to determine the correct correspondence
#https://forum.earthdata.nasa.gov/viewtopic.php?t=4161 for more info
img_vv = (
    ee.ImageCollection('COPERNICUS/S1_GRD')
    .filterBounds(loc)
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH'))
    .filter(ee.Filter.eq('instrumentMode', 'IW'))
    .filter(ee.Filter.eq('relativeOrbitNumber_start', path))
    .filter(ee.Filter.eq('sliceNumber', frame))
    .select('VH')
)

#reproject to lower resolution
def downsample(img):
    # Reduce resolution using mean reducer
    return img.resample('bilinear').reproject(
        crs='EPSG:32607',
        scale=500
    )

img_vv = img_vv.map(downsample)
# Extract distinct sliceNumbers for trial/error finding correct frame
slice_numbers = (img_vv
                 .aggregate_array("sliceNumber")
                 .distinct()
                 .sort())

data_window = ee.Filter.date('2019-01-01', '2020-01-01')
img_vv = img_vv.filter(data_window)

print("Distinct sliceNumbers:", slice_numbers.getInfo())
print("Number of images:", img_vv.size().getInfo())
numImgs = img_vv.size().getInfo()
vis_params = {
    'min': -25,
    'max': 5,
    'palette': ['blue', 'green', 'white']
}

grd_map = geemap.Map()
grd_map.addLayer(img_vv.first(), vis_params, 'GRD', True)
grd_map #give it time to load


Distinct sliceNumbers: [5]
Number of images: 31


Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], position='topright', transp…

In [6]:
#Get bounds of union of image collection data
footprint_union = img_vv.geometry().dissolve()
#reset Map
# Add the union footprint
grd_map.addLayer(footprint_union, {'color': 'red'}, 'S1 Footprint Union')
grd_map

Map(bottom=812.0, center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], position='top…

Exporting GRD

In [97]:
# Get size of collection
n = img_vv.size().getInfo()
# Convert collection to Python list of ee.Images
img_list = img_vv.toList(n)

for i in range(n):
    img = ee.Image(img_list.get(i))
    
    # Get system:index on the client
    img_id = img.get('system:index').getInfo()
    
    task = ee.batch.Export.image.toDrive(
        image=img,
        description=f'export_{img_id}',
        folder='GRD',  # optional, Drive folder name
        fileNamePrefix=f's1_500m_{img_id}',
        region=img.geometry(),
        scale=500,
        crs='EPSG:32607',
        fileFormat='GeoTIFF',
        maxPixels=1e13
    )
    task.start()
    print(f"Started export for {img_id}")

Started export for S1A_IW_GRDH_1SDV_20190101T155619_20190101T155644_025286_02CC00_8276
Started export for S1A_IW_GRDH_1SDV_20190113T155618_20190113T155643_025461_02D249_16A5
Started export for S1A_IW_GRDH_1SDV_20190125T155618_20190125T155643_025636_02D8AE_B476
Started export for S1A_IW_GRDH_1SDV_20190206T155617_20190206T155642_025811_02DEF5_FC07
Started export for S1A_IW_GRDH_1SDV_20190218T155617_20190218T155642_025986_02E534_8F06
Started export for S1A_IW_GRDH_1SDV_20190302T155617_20190302T155642_026161_02EB7D_B8C4
Started export for S1A_IW_GRDH_1SDV_20190314T155617_20190314T155642_026336_02F1E0_60CB
Started export for S1A_IW_GRDH_1SDV_20190326T155618_20190326T155643_026511_02F84D_429C
Started export for S1A_IW_GRDH_1SDV_20190407T155618_20190407T155643_026686_02FEC1_A9AD
Started export for S1A_IW_GRDH_1SDV_20190419T155618_20190419T155643_026861_03051A_E07F
Started export for S1A_IW_GRDH_1SDV_20190501T155619_20190501T155644_027036_030B79_403F
Started export for S1A_IW_GRDH_1SDV_2019051

Get the DEM

In [ ]:
#Get DEM of GRD union bounds:

dem_clipped = (
    ee.ImageCollection("COPERNICUS/DEM/GLO30") 
    .filterBounds(footprint_union)
    .select('DEM')
)

single_dem = dem_clipped.mosaic().toFloat()

#CMU colors
vis_params = {
    'min': 0,
    'max': 3000,
    'palette': ['blue', 'green', 'white']
}

#Sanity check
centroid = footprint_union.centroid().coordinates().getInfo()
lon, lat = centroid[0], centroid[1]
dem_map = geemap.Map()
dem_map.setCenter(lon, lat, zoom=5)
dem_map.addLayer(single_dem, vis_params, 'DEM')
dem_map

Map(center=[61.609210398328756, -143.1184445951888], controls=(WidgetControl(options=['position', 'transparent…

In [91]:
#process glacier files

#declare glacier id func
def add_numeric_id(f):
    rgi_id = ee.String(f.get("rgi_id"))  # e.g. "RGI2000-v7.0-G-11-00351"
     # Split on "-"
    parts = rgi_id.split("-")   # returns ee.List

    # Get region and number explicitly by index
    region = ee.String(parts.get(3))   # "11"
    number = ee.String(parts.get(4))   # "00351"

    # Convert to integer
    glac_id = ee.Number.parse(number)

    return f.set("glac_id", glac_id)



rgi_vector = (
    ee.FeatureCollection("projects/sat-io/open-datasets/RGI/RGI_VECTOR_MERGED_V7")
    .filterBounds(footprint_union)
)

rgi_with_id = rgi_vector.map(add_numeric_id)

rgi = rgi_with_id.reduceToImage(
    properties=["glac_id"],
    reducer=ee.Reducer.first()
).unmask(0) 

rgi_geo = rgi_vector.geometry().dissolve().simplify(maxError=250)
#Clip grd to glaciers
img_vv_clipped = img_vv.map(lambda img: img.clip(rgi_geo))

vis_params = {
    'min': 0,
    'max': 100000,
    'palette': ['black', 'green', 'white']
}
#rgi = rgi.clip(footprint_union)
GlacierMap = geemap.Map()
rgi_binary = rgi.gt(0).unmask()
GlacierMap.addLayer(rgi, vis_params, 'Glacier raster mask', True)
#GlacierMap.addLayer(foot,vis_params,'Glacier Features')
GlacierMap #takes a while to load all of them

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], position='topright', transp…


Sanity Checking

Generate video

In [13]:


video_args = {
  'dimensions': 768,
  'region': footprint_union,
  'framesPerSecond': 7,
  'crs': 'EPSG:3857',
  'min': -40.0,
  'max': 35.0,
  'palette': ['blue', 'purple', 'cyan', 'green', 'yellow', 'red']
}

animation_url = img_vv.filterDate('2019-01-01', '2020-01-02').getVideoThumbURL(video_args)
print(f"Animation URL: {animation_url}")

Animation URL: https://earthengine-highvolume.googleapis.com/v1/projects/sar-testing-472822/videoThumbnails/fb17dd3d57f74470dc921eeb57414d36-22c3d6c096da56cb41ef29a88a3d2c29:getPixels


Merge DEM and export data!

In [101]:
img = single_dem
task = ee.batch.Export.image.toDrive(
        image=img,
        description=f'export_dem',
        folder='GRD',  # optional, Drive folder name
        fileNamePrefix=f'DEM',
        region=footprint_union,
        scale=30,
        crs='EPSG:32607',
        fileFormat='GeoTIFF',
        maxPixels=1e13
    )
task.start()
print(f"Started export dem")

Started export dem


Reformatting:

In [7]:
#Create parameters
# Take first image in collection
img = img_vv.first()
# Get projection of a band (VV, VH, etc.)
proj = img.select(0).projection()
# CRS string (e.g. "EPSG:32607")
crs = proj.crs().getInfo()
print(f"CRS: {crs}")

bounds = footprint_union.bounds().coordinates().get(0).getInfo()

# bounds is a list of coordinates (a ring), so we extract the corners
xmin = int(np.min(bounds[0][0]))
ymin = int(np.min(bounds[0][1]))
xmax = int(np.max(bounds[2][0]))
ymax = int(np.max(bounds[2][1]))

print("xmin:", xmin)
print("ymin:", ymin)
print("xmax:", xmax)
print("ymax:", ymax)
# need bounds in different order for shapefile work below (minx,miny,maxx,maxy)
output_bounds = [xmin, ymin, xmax, ymax]

RGI_shapefile_path = "C:/Users/jaden/Downloads/Research/Glaciers/RGI2000-v7.0-G-01_alaska"
srcDS = gdal.OpenEx(RGI_shapefile_path)
layer = srcDS.GetLayer()
print(layer.GetName())


CRS: EPSG:32607
xmin: -145
ymin: 60
xmax: -140
ymax: 62
RGI2000-v7.0-G-01_alaska


In [8]:
in_gd_dem = gdal.Open("C:/Users/jaden/Downloads/Research/Notebooks/GEE/tifs/DEM.tif")  

spatial_ref = osr.SpatialReference()
spatial_ref.ImportFromWkt(in_gd_dem.GetProjection())

geotransform = in_gd_dem.GetGeoTransform()
top_left_x = geotransform[0]
pixel_width = geotransform[1]
top_left_y = geotransform[3]
pixel_height = geotransform[5]
            
# Get the number of columns and rows
cols = in_gd_dem.RasterXSize
rows = in_gd_dem.RasterYSize
            
# Calculate the extent (bounds)
xmin = top_left_x
xmax = top_left_x + (cols * pixel_width)
ymax = top_left_y
ymin = top_left_y + (rows * pixel_height)


print("xmin:", xmin)
print("ymin:", ymin)
print("xmax:", xmax)
print("ymax:", ymax)
# need bounds in different order for shapefile work below (minx,miny,maxx,maxy)
output_bounds = [xmin, ymin, xmax, ymax]
projWin=(xmin, ymax, xmax, ymin) 
                 

xmin: 246420.0
ymin: 6722850.0
xmax: 529290.0
ymax: 6941880.0


In [9]:
# %% 
# read RGI, make glacier id and aspect mask layers for cube
# 
# vector layer for gdal rasterize function
layers = ['RGI2000-v7.0-G-01_alaska',]

xres,yres = (500.0,500.0)   # output (datacube) resolution in projection meters
resampleAlg = 'average'     # resample technique


epsg_str_for_cube = crs
# set up field definition for integer value of rgi_id
fldDef = ogr.FieldDefn('rgi_int', ogr.OFTInteger)

srcDS = gdal.OpenEx(RGI_shapefile_path)
ds = gdal.VectorTranslate('',srcDS,format = 'Memory',spatFilter=output_bounds,
                          spatSRS=epsg_str_for_cube,reproject=True,dstSRS=epsg_str_for_cube)

layer = ds.GetLayer()
layer_defn = layer.GetLayerDefn()

print("Fields available in the layer:")
for i in range(layer_defn.GetFieldCount()):
    print("-", layer_defn.GetFieldDefn(i).GetName())



#get layer and add the field:
layer = ds.GetLayer()
layer.CreateField(fldDef)


for feat in layer:
    rgi_id_str = feat.GetField('rgi_id')
    rgi_int = int(rgi_id_str.split('-')[-1])
    feat.SetField('rgi_int',rgi_int) 
    layer.SetFeature(feat) 
    

outMemRas = '/vsimem/raster_rgi_ids.tif'

gdal.Rasterize(outMemRas, ds,
               outputType=gdal.GDT_UInt32,
               outputSRS=epsg_str_for_cube,
               xRes=xres,
               yRes=yres,
#                width=XSize, height=YSize,
               outputBounds=output_bounds,
               layers=layers,
               attribute='rgi_int',
#                allTouched=True
               )
shapeRasDS = gdal.Open(outMemRas) # Open raster file in memory with gdal
rgi_ind_shape_arr = shapeRasDS.ReadAsArray() # Read into numpy array
gdal.Unlink(outMemRas)
shapeRasDS = None



outMemRas = '/vsimem/raster_aspect.tif'
print(layers)
gdal.Rasterize(outMemRas, ds,
               outputType=gdal.GDT_Float32,
               outputSRS=epsg_str_for_cube,
               xRes=xres,
               yRes=yres,
#                width=XSize, height=YSize,
               outputBounds=output_bounds,
               layers=layers,
               attribute='aspect_deg',
#                allTouched=True
               )
shapeRasDS = gdal.Open(outMemRas) # Open raster file in memory with gdal
rgi_aspect_arr = shapeRasDS.ReadAsArray() # Read into numpy array
gdal.Unlink(outMemRas)
shapeRasDS = None



Fields available in the layer:
- rgi_id
- o1region
- o2region
- glims_id
- anlys_id
- subm_id
- src_date
- cenlon
- cenlat
- utm_zone
- area_km2
- primeclass
- conn_lvl
- surge_type
- term_type
- glac_name
- is_rgi6
- termlon
- termlat
- zmin_m
- zmax_m
- zmed_m
- zmean_m
- slope_deg
- aspect_deg
- aspect_sec
- dem_source
- lmax_m
['RGI2000-v7.0-G-01_alaska']


Load GRD into cube

In [ ]:
out_nc_fn = "out.nc"
in_gd_dem = gdal.Translate('',in_gd_dem,format='VRT',projWin=projWin,xRes=xres,yRes=yres,resampleAlg=resampleAlg)

min_x,pix_x_m,r1,max_y,r2,pix_y_m = in_gd_dem.GetGeoTransform()
    
x_vec_sm = (min_x + pix_x_m/2.0) + (pix_x_m * np.arange(0,in_gd_dem.RasterXSize))
y_vec_sm = (max_y + pix_y_m/2.0) + (pix_y_m * np.arange(0,in_gd_dem.RasterYSize))
    

time_vec = np.ones((numImgs),dtype='datetime64[ns]')
    
ny,nx = in_gd_dem.RasterYSize,in_gd_dem.RasterXSize





In [ ]:
img_cube = np.memmap('img_cube.dat', dtype='float32', mode='w+', shape=(numImgs, ny, nx))
img_cube[:] = np.nan